# Part 2 - SQL Challenge answers

In [ ]:
from sshtunnel import SSHTunnelForwarder

AWS_IP_ADDRESS = '52.27.27.155'
AWS_USERNAME = 'jason'
SSH_KEY_PATH = '/Users/jadams/.ssh/id_rsa'

server = SSHTunnelForwarder(
    AWS_IP_ADDRESS,
    ssh_username=AWS_USERNAME,
    ssh_pkey=SSH_KEY_PATH,
    remote_bind_address=('localhost', 5432),
)

server.start()
print(server.is_active, server.is_alive, server.local_bind_port)

##  Connecting via Python
We'll be using a Psycopg2 connector alongside SQLAlchemy to connect to this database.

* **SQLAlchemy:** generates SQL statements
* **Psycopg2:** sends the SQL statements to the Postgres database

    Let's make the connection to the database. Note that the IP address of the Postgres database is 'localhost' and the port is set to whatever the `server` connection above contains. This is because we have used the SSH tunnel to create a connection between the AWS instance and our computer. SSH tunnels enable remote instances to behave as if they are *local*.

In [1]:
from sqlalchemy import create_engine

# Postgres username, password, and database name
POSTGRES_IP_ADDRESS = 'localhost' ## This is localhost because SSH tunnel is active
POSTGRES_PORT = 5432 ## Running a tunnel in terminal instead of in the notebook
POSTGRES_USERNAME = 'jason'     ## CHANGE THIS TO YOUR POSTGRES USERNAME
POSTGRES_PASSWORD = 'jason' ## CHANGE THIS TO YOUR POSTGRES PASSWORD
POSTGRES_DBNAME = 'baseball'

# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'
                .format(username=POSTGRES_USERNAME, 
                        password=POSTGRES_PASSWORD,
                        ipaddress=POSTGRES_IP_ADDRESS,
                        port=POSTGRES_PORT,
                        dbname=POSTGRES_DBNAME))

# Create the connection
cnx = create_engine(postgres_str)

### Question 1
1. What was the total spent on salaries by each team, each year?

In [2]:
import pandas as pd

In [4]:
pd.read_sql_query('''SELECT yearid, teamid, sum(salary) as total_salary 
                        FROM salaries 
                        GROUP BY yearid, teamid
                        ORDER by teamid, yearid;''', cnx).head()

,yearid,teamid,total_salary
0,1997,ANA,31135472.0
1,1998,ANA,41281000.0
2,1999,ANA,55388166.0
3,2000,ANA,51464167.0
4,2001,ANA,47535167.0


### Question 2
2. What is the first and last year played for each player? *Hint:* Create a new table from 'Fielding.csv'.

Get the CSV files locally and take a look at the head of Fielding.csv

In [23]:
! wget http://seanlahman.com/files/database/lahman-csv_2014-02-14.zip -P 'baseball/'


--2018-10-20 13:50:04--  http://seanlahman.com/files/database/lahman-csv_2014-02-14.zip
Resolving seanlahman.com (seanlahman.com)... 69.163.161.183
Connecting to seanlahman.com (seanlahman.com)|69.163.161.183|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://www.seanlahman.com/files/database/lahman-csv_2014-02-14.zip [following]
--2018-10-20 13:50:04--  http://www.seanlahman.com/files/database/lahman-csv_2014-02-14.zip
Resolving www.seanlahman.com (www.seanlahman.com)... 69.163.161.183
Reusing existing connection to seanlahman.com:80.
HTTP request sent, awaiting response... 200 OK
Length: 8640782 (8.2M) [application/zip]
Saving to: ‘baseball/lahman-csv_2014-02-14.zip’

lahman-csv_2014-02- 100%[===================>]   8.24M  7.45MB/s    in 1.1s    

2018-10-20 13:50:05 (7.45 MB/s) - ‘baseball/lahman-csv_2014-02-14.zip’ saved [8640782/8640782]



In [25]:
! unzip baseball/lahman-csv_2014-02-14.zip -d 'baseball/'

Archive:  baseball/lahman-csv_2014-02-14.zip
  inflating: baseball/SchoolsPlayers.csv  
  inflating: baseball/SeriesPost.csv  
  inflating: baseball/Teams.csv      
  inflating: baseball/TeamsFranchises.csv  
  inflating: baseball/TeamsHalf.csv  
  inflating: baseball/AllstarFull.csv  
  inflating: baseball/Appearances.csv  
  inflating: baseball/AwardsManagers.csv  
  inflating: baseball/AwardsPlayers.csv  
  inflating: baseball/AwardsShareManagers.csv  
  inflating: baseball/AwardsSharePlayers.csv  
  inflating: baseball/Batting.csv    
  inflating: baseball/BattingPost.csv  
  inflating: baseball/Fielding.csv   
  inflating: baseball/FieldingOF.csv  
  inflating: baseball/FieldingPost.csv  
  inflating: baseball/HallOfFame.csv  
  inflating: baseball/Managers.csv   
  inflating: baseball/ManagersHalf.csv  
  inflating: baseball/Master.csv     
  inflating: baseball/Pitching.csv   
  inflating: baseball/PitchingPost.csv  
  inflating: baseball/readme2013.txt  
  inflating: baseball/S

In [26]:
! cat baseball/Fielding.csv | head











cat: stdout: Broken pipe


In [7]:
def build_sql_table(table_name):
    df = pd.read_csv(f'baseball/{table_name}.csv')
    df.columns = df.columns.str.lower()

    df[:0].to_sql(table_name.lower(), cnx, index=False, if_exists='replace')

In [8]:
build_sql_table('Fielding')

In [9]:
! psql -d baseball -h localhost -U jason -c "COPY fielding FROM '/home/jason/baseballdata/Fielding.csv' DELIMITER ',' CSV HEADER;"

COPY 166991


In [10]:
pd.read_sql(''' SELECT playerid, MIN(yearid) as first_year, MAX(yearid) as last_year
                FROM fielding
                GROUP BY playerid;
            ''', cnx).head()

,playerid,first_year,last_year
0,gagnied01,1914,1915
1,ramospe01,1955,1970
2,putkolu01,2012,2013
3,boyerke01,1955,1969
4,dillibo01,1946,1951


### Question 3
3. Who has played the most all star games?

In [27]:
!psql -h localhost -U jason -d baseball -c "\d"

          List of relations
 Schema |    Name     | Type  | Owner 
--------+-------------+-------+-------
 public | allstarfull | table | jason
 public | fielding    | table | jason
 public | franchises  | table | jason
 public | salaries    | table | jason
 public | schools     | table | jason
(5 rows)



In [32]:
pd.read_sql(''' SELECT playerid, count(1) as all_star_game_count 
                FROM allstarfull 
                GROUP BY playerid
                ORDER BY all_star_game_count DESC
                LIMIT 1;''', cnx)

,playerid,all_star_game_count
0,aaronha01,25


### Question 4
4. Which school has generated the most distinct players? *Hint:* Create new table from 'CollegePlaying.csv'.

In [27]:
build_sql_table('SchoolsPlayers')

! psql -d baseball -h localhost -U jason -c "COPY schoolsplayers FROM '/home/jason/baseballdata/SchoolsPlayers.csv' DELIMITER ',' CSV HEADER;"

In [20]:
pd.read_sql("""
            SELECT  schoolid
                    , count(1) as num_players 
            FROM schoolsplayers
            GROUP BY schoolid
            ORDER BY num_players DESC
            LIMIT 1;
            """,
           cnx).head()

,schoolid,num_players
0,usc,102


### Question 5

5. Which players have the longest career? Assume that the `debut` and `finalGame` columns comprise the start and end, respectively, of a player's career. *Hint:* Create a new table from 'Master.csv'. Also note that strings can be converted to dates using the [`DATE`](https://wiki.postgresql.org/wiki/Working_with_Dates_and_Times_in_PostgreSQL#WORKING_with_DATETIME.2C_DATE.2C_and_INTERVAL_VALUES) function and can then be subtracted from each other yielding their difference in days.

In [26]:
build_sql_table('Master')

! psql -d baseball -h localhost -U jason -c "COPY master FROM '/home/jason/baseballdata/Master.csv' DELIMITER ',' CSV HEADER;"

COPY 18354


In [42]:
pd.read_sql("""
            SELECT  namegiven, 
                    namelast, 
                    DATE(finalgame) - DATE(debut) AS career_length, 
                    debut, 
                    finalgame 
            FROM master
            WHERE debut <> 'None'
            OR    finalgame <> 'None'
            ORDER BY career_length DESC
            LIMIT 5;
            """
           , cnx).head()

,namegiven,namelast,career_length,debut,finalgame
0,Nicholas,Altrock,12862,1898-07-14,1933-10-01
1,James Henry,O'Rourke,11836,1872-04-26,1904-09-22
2,Saturnino Orestes Armas,Minoso,11492,1949-04-19,1980-10-05
3,Charles Timothy,O'Leary,11126,1904-04-14,1934-09-30
4,Walter Arlington,Latham,10678,1880-07-05,1909-09-30


### Question 6

6. What is the distribution of debut months? *Hint:* Look at the `DATE` and [`EXTRACT`](https://www.postgresql.org/docs/current/static/functions-datetime.html#FUNCTIONS-DATETIME-EXTRACT) functions.

In [53]:
pd.read_sql("""
            SELECT  EXTRACT(MONTH FROM DATE(debut)) AS debut_month,
                    COUNT(1) as players_debut
            FROM master
            WHERE debut <> 'None'
            GROUP BY debut_month
            ORDER BY debut_month;
            """
           , cnx)

,debut_month,players_debut
0,3.0,41
1,4.0,4711
2,5.0,2230
3,6.0,1893
4,7.0,1978
5,8.0,1943
6,9.0,5061
7,10.0,308


### Question 7

7. What is the effect of table join order on mean salary for the players listed in the main (master) table? *Hint:* Perform two different queries, one that joins on playerID in the salary table and other that joins on the same column in the master table. You will have to use left joins for each since right joins are not currently supported with SQLalchemy.

In [56]:
build_sql_table('Salaries')

! psql -d baseball -h localhost -U jason -c "COPY salaries FROM '/home/jason/baseballdata/Salaries.csv' DELIMITER ',' CSV HEADER;"

COPY 23956


In [70]:
pd.read_sql("""
            SELECT AVG(salary)
            FROM salaries as s
            LEFT JOIN master as m
                ON m.playerid = s.playerid;
            """
           , cnx)

,avg
0,1.864357e+06


In [60]:
pd.read_sql("""
            SELECT  AVG(s.salary)
            FROM master as m
            LEFT JOIN salaries as s 
                ON m.playerid = s.playerid;
            """
           , cnx)

,avg
0,1.864357e+06


In [72]:
pd.read_sql("""
            SELECT  AVG(s.salary)
            FROM master as m
            INNER JOIN salaries as s 
                ON m.playerid = s.playerid;
            """
           , cnx)

,avg
0,1.864357e+06


The avegage salary when performing a `LEFT JOIN` from `master` to `salaries` is the same as performing a `LEFT JOIN` from `salaries` to `master`. This means the same playerids exist in both tables.